In [4]:
import os
import rasterio
import geopandas as gpd
from rasterio.mask import mask
from rasterio.features import shapes
import tools  # your custom GIS functions
import sys
import numpy as np
sys.path.append('/Applications/WhiteboxTools_darwin_m_series/WBT')  # Path to whitebox_tools.py
from whitebox_tools import WhiteboxTools
wbt = WhiteboxTools()
import matplotlib.pyplot as plt
from shapely.geometry import shape
from scipy import ndimage

# PARAMETERS
catchment_no = np.arange(1,23,1)
exclude = [11, 17, 18, 19]
catchs = np.setdiff1d(catchment_no, exclude)
resos = [20, 40, 80, 160]

# OUTPUT ROOT DIR
local_output_root = '/Users/jpnousu/Krycklan_GIS_data'

# shapefiles
streams_fp = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/5haStreams.shp'
ditches_fp = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/ManuallyDigitizedDitchesKrycklan.shp'
roads_fp = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/TOPOGRAFI_50/shapefiles/combined_roads.shp'
catchment_dir = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_catchments/split_catchments'

# DEM FILE PATH
dem_original_fp = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/Krycklan_2015_DEM_0.5m/Krycklan_2015_DEM_cleaned.tif'

# Base names for outputs
streams_name = os.path.splitext(os.path.basename(streams_fp))[0]
ditches_name = os.path.splitext(os.path.basename(ditches_fp))[0]
roads_name = os.path.splitext(os.path.basename(roads_fp))[0]

# Loop through catchments (and resolutions)
for reso in resos:
    # Reference file path
    ref_fp = f'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/FOREST_BASIC_KRYCKLAN/processed/canopy_height_{reso}.asc'
    dem_match_fp = f'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/processed/Krycklan_2015_DEM_match_{reso}.tif'
    # --- Step: Reproject and match DEM once ---
    tools.reproj_match(
        infile=dem_original_fp,
        match=ref_fp,
        outfile=dem_match_fp,
        resampling_method='average',
        save_in='tif')
    
    for catch in catchs:
        # Catchment shapefile for clipping
        cmask_shape = os.path.join(catchment_dir, f"catchment_{catch}.shp")
        print(f"\nProcessing catch={catch}, reso={reso}...")
        # Make dir
        out_dir = os.path.join(local_output_root, f"{reso}m", f"C{catch}")
        os.makedirs(out_dir, exist_ok=True)
        # Rasterized shapefiles
        out_fp_streams = os.path.join(out_dir, streams_name + ".asc")
        out_fp_ditches = os.path.join(out_dir, ditches_name + ".asc")
        combined_fp = os.path.join(out_dir, "channels.asc")
        # Clipped shapefiles
        streams_clipped_fp = os.path.join(out_dir, streams_name + "_clip.shp")
        roads_clipped_fp = os.path.join(out_dir, roads_name + "_clip.shp")
    
        ''' Read and clip the DEM '''
        # Read matched DEM and clip for this catchment
        with rasterio.open(dem_match_fp) as src:
            profile = src.profile
            transform = src.transform
            arr = src.read(1)
            nodata_val = src.nodata if src.nodata is not None else -9999
    
            profile.update({
                'driver': 'GTiff',
                'dtype': arr.dtype,
                'compress': 'lzw',
                'nodata': nodata_val
            })
    
        # --- Clip DEM to catchment ---
        gdf_catch = gpd.read_file(cmask_shape)
        gdf_catch = gdf_catch.to_crs(profile['crs'])
    
        with rasterio.open(dem_match_fp) as src:
            out_image, out_transform = mask(src, gdf_catch.geometry, crop=True, nodata=nodata_val)
            out_meta = src.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform,
                "nodata": nodata_val
            })

        # Mask those edge cells that do not have 4 neighbours (E,W,N,S) and 'islands'
        dem = out_image[0]
        # Boolean mask of valid cells
        valid = dem != nodata_val        # nodata_val came from src.nodata earlier
        # Remove isolated single pixels (no N, S, E, or W neighbour)
        kernel = np.array([[0, 1, 0],
                           [1, 0, 1],
                           [0, 1, 0]], dtype=int)
        neigh_count = ndimage.convolve(valid.astype(int), kernel, mode="constant", cval=0)
        isolated = valid & (neigh_count == 0)         # True where pixel is lonely
        valid[isolated] = False                       # drop them
        # Keep only the largest 4‑connected component
        labeled, ncomp = ndimage.label(valid, structure=[[0,1,0],
                                                         [1,1,1],
                                                         [0,1,0]])  # 4‑conn
        if ncomp > 1:
            # find sizes of each component
            sizes = np.bincount(labeled.ravel())
            sizes[0] = 0                              # background = 0, ignore
            biggest = sizes.argmax()                  # id of largest component
            valid &= (labeled == biggest)             # keep only the biggest
        # Apply the cleaned mask back to DEM and write
        clean_dem = np.where(valid, dem, nodata_val).astype(dem.dtype)
        out_image[0] = clean_dem                      # put back into 3‑D array
        clipped_dem_fp = os.path.join(out_dir, f"dem_clip.tif")
        with rasterio.open(clipped_dem_fp, 'w', **out_meta) as dest:
            dest.write(out_image)

        ''' Saving the cmask '''
        cmask_fp = os.path.join(out_dir, "cmask.tif")  # Output file path
        cmask_asc_fp = os.path.join(out_dir, "cmask.asc")  # ASCII output file path
        cmask_bi_asc_fp = os.path.join(out_dir, "cmask_bi.asc") 
        
        with rasterio.open(clipped_dem_fp) as src:
            data = src.read(1)
            meta = src.meta.copy()
        
            # Set desired output nodata value
            nodata_val = -9999.0
        
            # Create mask: 1.0 where data is NOT original nodata, -9999 where it was nodata
            cmask = np.where(data != src.nodata, catch, nodata_val)
            cmask_bi = np.where(data != src.nodata, 1, nodata_val)
            
            # Update metadata
            meta.update({
                "dtype": "float32",
                "nodata": nodata_val
            })
        
            # --- Save GeoTIFF ---
            with rasterio.open(cmask_fp, "w", **meta) as dst:
                dst.write(cmask.astype(np.float32), 1)
                
            # --- Save ASCII Grid ---
            asc_meta = meta.copy()
            asc_meta.update({
                "driver": "AAIGrid"
            })
            with rasterio.open(cmask_asc_fp, "w", **asc_meta) as dst:
                dst.write(cmask.astype(np.float32), 1)
            with rasterio.open(cmask_bi_asc_fp, "w", **asc_meta) as dst:
                dst.write(cmask_bi.astype(np.float32), 1)
    
        ''' Rasterize stream and ditch shapefiles '''
        tools.rasterize_shapefile(
            shapefile=streams_fp,
            burn_field='GRID_CODE',
            subset=None,
            out_fp=out_fp_streams,
            ref_raster=clipped_dem_fp,
            plot=False,
            save_in='asc'
        )
    
        tools.rasterize_shapefile(
            shapefile=ditches_fp,
            burn_field='VALUE',
            subset=None,
            out_fp=out_fp_ditches,
            ref_raster=clipped_dem_fp,
            plot=False,
            save_in='asc'
        )
    
        # --- Combine stream and ditch raster layers ---
        tools.fill_layer_na_with_layer(
            priority_layer=out_fp_streams,
            secondary_layer=out_fp_ditches,
            out_fp=combined_fp,
            save_in='asc'
        )
    
        ''' Load DEM and extract valid data region and clip shapefiles '''
        with rasterio.open(clipped_dem_fp) as src:
            dem = src.read(1)
            mask_area = dem != src.nodata  # where data exists
            transform = src.transform
            dem_crs = src.crs
            # Extract shapes (polygons) from valid data mask
            shapes_gen = shapes(source=np.ones_like(dem, dtype='uint8'), mask=mask_area, transform=transform)
            # Collect valid polygons
            polygons = [shape(geom) for geom, value in shapes_gen if value == 1]
        
        # Combine into a single polygon (dissolve)
        footprint = gpd.GeoDataFrame(geometry=polygons, crs=dem_crs)
        footprint = footprint.dissolve()  # dissolve to one multipolygon
        
        # Save as shapefile
        dem_extent_shp_fp = os.path.join(out_dir, "dem_extent.shp")
        footprint.to_file(dem_extent_shp_fp)
    
        # For clipping streams and roads
        extent_gdf = gpd.read_file(dem_extent_shp_fp)
        
        # Clip streams
        streams_gdf = gpd.read_file(streams_fp).to_crs(dem_crs)
        streams_clipped = gpd.clip(streams_gdf, extent_gdf)
        streams_clipped.to_file(streams_clipped_fp)
        
        # Clip roads
        roads_gdf = gpd.read_file(roads_fp).to_crs(dem_crs)
        roads_clipped = gpd.clip(roads_gdf, extent_gdf)
        roads_clipped.to_file(roads_clipped_fp)
        
        ''' WhiteboxTools DEM processing '''
        dem_base = os.path.splitext(clipped_dem_fp)[0]
        dir_base = os.path.dirname(dem_base)

        # Burn streams at roads
        dem_culverts_path = f"{dem_base}_culv.tif"
        wbt.burn_streams_at_roads(
            clipped_dem_fp,
            streams_clipped_fp,
            roads_clipped_fp,
            dem_culverts_path,
            width=50.,
        )

        # --- Fill nodata holes in dem_culverts_path using dem_clipped.tif as source ---
        dem_culverts_filled_path = f"{dem_base}_culv_fill.tif"
        
        with rasterio.open(dem_culverts_path) as culv_src, rasterio.open(clipped_dem_fp) as clipped_src:
            culv_dem = culv_src.read(1).astype(np.float32)
            clipped_dem = clipped_src.read(1).astype(np.float32)
        
            culv_nodata = culv_src.nodata if culv_src.nodata is not None else -9999.0
            clipped_nodata = clipped_src.nodata if clipped_src.nodata is not None else -9999.0
        
            # Mask for filling
            fill_mask = (culv_dem == culv_nodata) & (clipped_dem != clipped_nodata)
            culv_dem[fill_mask] = clipped_dem[fill_mask]
        
            profile = culv_src.profile
            profile.update({
                'dtype': 'float32',
                'nodata': culv_nodata,
                'compress': 'lzw'
            })
        
        # Write to new file
        with rasterio.open(dem_culverts_filled_path, 'w', **profile) as dst:
            dst.write(culv_dem, 1)
            
        dem_culverts_path = dem_culverts_filled_path

        # Rasterize water bodies (streams only)
        water_file = tools.rasterize_water_bodies(
            stream_file=streams_clipped_fp,
            ref_raster=dem_culverts_path,
        )
    
        # Burn water into DEM
        dem_culverts_stream_path = tools.burn_water_into_dem(
            dem_culverts_path,
            water_file,
            drop=1.0,
        )
    
        # Breach depressions
        dem_breached_path = f"{dem_base}_no_deps.tif"
        wbt.breach_depressions_least_cost(
            dem_culverts_stream_path,
            dem_breached_path,
            dist=1000,
            max_cost=1000.0,
        )
    
        # Slope
        slope_path = f"{dir_base}/slope.tif"
        wbt.slope(
            dem_breached_path,
            slope_path,
            units="degrees",
        )
    
        # Flow direction D8
        flow_pointer_path_d8 = f"{dir_base}/flowp_d8.tif"
        wbt.d8_pointer(
            dem_breached_path,
            flow_pointer_path_d8,
        )
    
        # Flow direction Dinf
        flow_pointer_path_dinf = f"{dir_base}/flowp_dinf.tif"
        wbt.d_inf_pointer(
            dem_breached_path,
            flow_pointer_path_dinf,
        )
    
        # Flow accumulation D8 SCA
        flow_acc_sca_path_d8 = f"{dir_base}/flowacc_sca_d8.tif"
        wbt.d8_flow_accumulation(
            dem_breached_path,
            flow_acc_sca_path_d8,
            out_type="specific contributing area",
        )
    
        # Flow accumulation Dinf SCA
        flow_acc_ca_path_dinf = f"{dir_base}/flowacc_sca_dinf.tif"
        wbt.d_inf_flow_accumulation(
            dem_breached_path,
            flow_acc_ca_path_dinf,
            out_type="specific contributing area",
        )

        # Flow accumulation D8 CA
        flow_acc_ca_path_d8 = f"{dir_base}/flowacc_ca_d8.tif"
        wbt.d8_flow_accumulation(
            dem_breached_path,
            flow_acc_ca_path_d8,
            out_type="catchment area",
        )
    
        # Flow accumulation Dinf CA
        flow_acc_sca_path_dinf = f"{dir_base}/flowacc_ca_dinf.tif"
        wbt.d_inf_flow_accumulation(
            dem_breached_path,
            flow_acc_sca_path_dinf,
            out_type="catchment area",
        )
        
        # TWI D8
        twi_d8_path = f"{dir_base}/twi_d8.tif"
        wbt.wetness_index(
            flow_acc_sca_path_d8,
            slope_path,
            twi_d8_path,
        )
    
        # TWI Dinf
        twi_dinf_path = f"{dir_base}/twi_dinf.tif"
        wbt.wetness_index(
            flow_acc_sca_path_dinf,
            slope_path,
            twi_dinf_path,
        )

        files_to_asc = ['dem_clip_culv_fill_streams.tif',
                        'twi_d8.tif',
                        'twi_dinf.tif',
                        'dem_clip_no_deps.tif'
                       ]
        
        # assumes you already have dir_base from earlier
        for tif_name in files_to_asc:
            tif_path = os.path.join(dir_base, tif_name)
            if not os.path.exists(tif_path):
                print(f"⚠️ File not found, skipping: {tif_path}")
                continue
        
            asc_name = os.path.splitext(tif_name)[0] + ".asc"
            asc_path = os.path.join(dir_base, asc_name)
        
            with rasterio.open(tif_path) as src:
                data = src.read(1)
                original_nodata = src.nodata
        
                # standardize nodata
                if original_nodata is not None:
                    data = np.where(data == original_nodata, -9999, data)
                else:
                    data = np.where(data == 0, -9999, data)
        
                profile = src.profile.copy()
                profile.update({
                    "driver": "AAIGrid",
                    "dtype": "float32",
                    "nodata": -9999,
                    "count": 1,
                })
        
                with rasterio.open(asc_path, "w", **profile) as dst:
                    dst.write(data, 1)
    
        print(f"WhiteboxTools processing done for catch={catch}, reso={reso}")

Coregistered to shape: 575 515 
 Affine | 20.00, 0.00, 726726.00|
| 0.00,-20.00, 7138690.00|
| 0.00, 0.00, 1.00|

Processing catch=1, reso=20...
./whitebox_tools --run="BurnStreamsAtRoads" --wd="/Users/jpnousu/WBT_data/" --dem='/Users/jpnousu/Krycklan_GIS_data/20m/C1/dem_clip.tif' --streams='/Users/jpnousu/Krycklan_GIS_data/20m/C1/5haStreams_clip.shp' --roads='/Users/jpnousu/Krycklan_GIS_data/20m/C1/combined_roads_clip.shp' --output='/Users/jpnousu/Krycklan_GIS_data/20m/C1/dem_clip_culv.tif' --width='50.0' -v --compress_rasters=False

*********************************
* Welcome to BurnStreamsAtRoads *
* Powered by WhiteboxTools      *
* www.whiteboxgeo.com           *
*********************************
Reading streams and roads data...
Reading DEM raster...
Rasterizing Streams: 12%
Rasterizing Streams: 25%
Rasterizing Streams: 37%
Rasterizing Streams: 50%
Rasterizing Streams: 62%
Rasterizing Streams: 75%
Rasterizing Streams: 87%
Rasterizing Streams: 100%
Rasterizing Streams: 112%
Findin

/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_83022/2281483216.py:214: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: PROJCS["SWEREF99 TM",GEOGCS["SWEREF99",DATUM["SWER ...
Right CRS: EPSG:3006

  streams_clipped = gpd.clip(streams_gdf, extent_gdf)
/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_83022/2281483216.py:219: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: PROJCS["SWEREF99 TM",GEOGCS["SWEREF99",DATUM["SWER ...
Right CRS: EPSG:3006

  roads_clipped = gpd.clip(roads_gdf, extent_gdf)
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field skapad create as date field, though DateTime requested.
  ogr_write(


Elapsed Time (excluding I/O): 0.1s
./whitebox_tools --run="Slope" --wd="/Users/jpnousu/WBT_data/" --dem='/Users/jpnousu/Krycklan_GIS_data/20m/C1/dem_clip_no_deps.tif' --output='/Users/jpnousu/Krycklan_GIS_data/20m/C1/slope.tif' --units=degrees -v --compress_rasters=False

****************************
* Welcome to Slope         *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 4%
Progress: 5%
Progress: 7%
Progress: 8%
Progress: 10%
Progress: 11%
Progress: 13%
Progress: 14%
Progress: 16%
Progress: 17%
Progress: 19%
Progress: 20%
Progress: 22%
Progress: 23%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 29%
Progress: 30%
Progress: 32%
Progress: 33%
Progress: 35%
Progress: 36%
Progress: 38%
Progress: 39%
Progress: 41%
Progress: 42%
Progress: 44%
Progress: 45%
Progress: 47%
Progress: 48%
Progress: 50%
Progress: 51%
Progress: 52%
Progress: 54%
Progress: 55%
Progress: 57%
Progress: 58%
Progress: 60%
P

/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_83022/2281483216.py:214: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: PROJCS["SWEREF99 TM",GEOGCS["SWEREF99",DATUM["SWER ...
Right CRS: EPSG:3006

  streams_clipped = gpd.clip(streams_gdf, extent_gdf)
/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_83022/2281483216.py:219: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: PROJCS["SWEREF99 TM",GEOGCS["SWEREF99",DATUM["SWER ...
Right CRS: EPSG:3006

  roads_clipped = gpd.clip(roads_gdf, extent_gdf)
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field skapad create as date field, though DateTime requested.
  ogr_write(


./whitebox_tools --run="Slope" --wd="/Users/jpnousu/WBT_data/" --dem='/Users/jpnousu/Krycklan_GIS_data/20m/C2/dem_clip_no_deps.tif' --output='/Users/jpnousu/Krycklan_GIS_data/20m/C2/slope.tif' --units=degrees -v --compress_rasters=False

****************************
* Welcome to Slope         *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Progress: 0%
Progress: 2%
Progress: 5%
Progress: 7%
Progress: 10%
Progress: 13%
Progress: 15%
Progress: 18%
Progress: 21%
Progress: 23%
Progress: 26%
Progress: 28%
Progress: 31%
Progress: 34%
Progress: 36%
Progress: 39%
Progress: 42%
Progress: 44%
Progress: 47%
Progress: 50%
Progress: 52%
Progress: 55%
Progress: 57%
Progress: 60%
Progress: 63%
Progress: 65%
Progress: 68%
Progress: 71%
Progress: 73%
Progress: 76%
Progress: 78%
Progress: 81%
Progress: 84%
Progress: 86%
Progress: 89%
Progress: 92%
Progress: 94%
Progress: 97%
Progress: 100%
Saving data...
Output file written
Elapsed Time (excluding I/O): 0.3s
./wh

...

./whitebox_tools --run="Slope" --wd="/Users/jpnousu/WBT_data/" --dem='/Users/jpnousu/Krycklan_GIS_data/160m/C20/dem_clip_no_deps.tif' --output='/Users/jpnousu/Krycklan_GIS_data/160m/C20/slope.tif' --units=degrees -v --compress_rasters=False

****************************
* Welcome to Slope         *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Progress: 0%
Progress: 12%
Progress: 25%
Progress: 37%
Progress: 50%
Progress: 62%
Progress: 75%
Progress: 87%
Progress: 100%
Saving data...
Output file written
Elapsed Time (excluding I/O): 0.1s
./whitebox_tools --run="D8Pointer" --wd="/Users/jpnousu/WBT_data/" --dem='/Users/jpnousu/Krycklan_GIS_data/160m/C20/dem_clip_no_deps.tif' --output='/Users/jpnousu/Krycklan_GIS_data/160m/C20/flowp_d8.tif' -v --compress_rasters=False

****************************
* Welcome to D8Pointer     *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Progress: 0%
Progress: 

/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_83022/2281483216.py:214: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: PROJCS["SWEREF99 TM",GEOGCS["SWEREF99",DATUM["SWER ...
Right CRS: EPSG:3006

  streams_clipped = gpd.clip(streams_gdf, extent_gdf)
/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_83022/2281483216.py:219: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: PROJCS["SWEREF99 TM",GEOGCS["SWEREF99",DATUM["SWER ...
Right CRS: EPSG:3006

  roads_clipped = gpd.clip(roads_gdf, extent_gdf)
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field skapad create as date field, though DateTime requested.
  ogr_write(


./whitebox_tools --run="Slope" --wd="/Users/jpnousu/WBT_data/" --dem='/Users/jpnousu/Krycklan_GIS_data/160m/C21/dem_clip_no_deps.tif' --output='/Users/jpnousu/Krycklan_GIS_data/160m/C21/slope.tif' --units=degrees -v --compress_rasters=False

****************************
* Welcome to Slope         *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Progress: 0%
Progress: 16%
Progress: 33%
Progress: 50%
Progress: 66%
Progress: 83%
Progress: 100%
Saving data...
Output file written
Elapsed Time (excluding I/O): 0.0s
./whitebox_tools --run="D8Pointer" --wd="/Users/jpnousu/WBT_data/" --dem='/Users/jpnousu/Krycklan_GIS_data/160m/C21/dem_clip_no_deps.tif' --output='/Users/jpnousu/Krycklan_GIS_data/160m/C21/flowp_d8.tif' -v --compress_rasters=False

****************************
* Welcome to D8Pointer     *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Progress: 0%
Progress: 16%
Progress: 33%
Progress: 

/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_83022/2281483216.py:214: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: PROJCS["SWEREF99 TM",GEOGCS["SWEREF99",DATUM["SWER ...
Right CRS: EPSG:3006

  streams_clipped = gpd.clip(streams_gdf, extent_gdf)
/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_83022/2281483216.py:219: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: PROJCS["SWEREF99 TM",GEOGCS["SWEREF99",DATUM["SWER ...
Right CRS: EPSG:3006

  roads_clipped = gpd.clip(roads_gdf, extent_gdf)
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field skapad create as date field, though DateTime requested.
  ogr_write(


./whitebox_tools --run="Slope" --wd="/Users/jpnousu/WBT_data/" --dem='/Users/jpnousu/Krycklan_GIS_data/160m/C22/dem_clip_no_deps.tif' --output='/Users/jpnousu/Krycklan_GIS_data/160m/C22/slope.tif' --units=degrees -v --compress_rasters=False

****************************
* Welcome to Slope         *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Progress: 0%
Progress: 7%
Progress: 15%
Progress: 23%
Progress: 30%
Progress: 38%
Progress: 46%
Progress: 53%
Progress: 61%
Progress: 69%
Progress: 76%
Progress: 84%
Progress: 92%
Progress: 100%
Saving data...
Output file written
Elapsed Time (excluding I/O): 0.0s
./whitebox_tools --run="D8Pointer" --wd="/Users/jpnousu/WBT_data/" --dem='/Users/jpnousu/Krycklan_GIS_data/160m/C22/dem_clip_no_deps.tif' --output='/Users/jpnousu/Krycklan_GIS_data/160m/C22/flowp_d8.tif' -v --compress_rasters=False

****************************
* Welcome to D8Pointer     *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *

In [5]:
import os
import rasterio
import numpy as np

# Path to directory containing merged TIFFs
all_dir = '/Users/jpnousu/Krycklan_GIS_data/25m/ALL'

# List all .tif files in the ALL directory
tif_files = [f for f in os.listdir(all_dir) if f.endswith('.tif')]

for tif_file in tif_files:
    tif_path = os.path.join(all_dir, tif_file)
    
    # Set output .asc filename
    asc_filename = os.path.splitext(tif_file)[0] + '.asc'
    asc_path = os.path.join(all_dir, asc_filename)

    # Read the raster
    with rasterio.open(tif_path) as src:
        data = src.read(1)
        original_nodata = src.nodata

        # Replace any existing nodata values with -9999
        if original_nodata is not None:
            data = np.where(data == original_nodata, -9999, data)
        else:
            # Optionally: also treat zeros as nodata (comment out if not wanted)
            data = np.where(data == 0, -9999, data)

        # Build output profile
        profile = src.profile.copy()
        profile.update({
            'driver': 'AAIGrid',
            'dtype': 'float32',
            'nodata': -9999,
            'count': 1,
        })

        # Write as ASCII grid
        with rasterio.open(asc_path, 'w', **profile) as dst:
            dst.write(data, 1)

    print(f"✓ Saved with nodata=-9999: {asc_path}")

✓ Saved with nodata=-9999: /Users/jpnousu/Krycklan_GIS_data/25m/ALL/dem_clipped_no_deps_merged.asc
✓ Saved with nodata=-9999: /Users/jpnousu/Krycklan_GIS_data/25m/ALL/dem_clipped_culverts_filled_merged.asc
✓ Saved with nodata=-9999: /Users/jpnousu/Krycklan_GIS_data/25m/ALL/dem_clipped_twi_dinf_merged.asc
✓ Saved with nodata=-9999: /Users/jpnousu/Krycklan_GIS_data/25m/ALL/dem_clipped_flowacc_d8_merged.asc
✓ Saved with nodata=-9999: /Users/jpnousu/Krycklan_GIS_data/25m/ALL/dem_clipped_twi_d8_merged.asc
✓ Saved with nodata=-9999: /Users/jpnousu/Krycklan_GIS_data/25m/ALL/dem_clipped_flowacc_dinf_merged.asc
✓ Saved with nodata=-9999: /Users/jpnousu/Krycklan_GIS_data/25m/ALL/dem_clipped_culverts_merged.asc
✓ Saved with nodata=-9999: /Users/jpnousu/Krycklan_GIS_data/25m/ALL/dem_clipped_slope_merged.asc
✓ Saved with nodata=-9999: /Users/jpnousu/Krycklan_GIS_data/25m/ALL/dem_clipped_merged.asc
